In [1]:
import os
import torch
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, models, datasets
import torch.optim as optim
import torch.nn as nn
from sklearn.model_selection import KFold
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, precision_recall_fscore_support)
from transformers import ViTForImageClassification, ViTFeatureExtractor
import optuna
from torchvision.datasets import ImageFolder

In [3]:
dataset_dir = '/kaggle/input/deepfake/DeepFake'

In [4]:
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),  
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(p=0.2),
    transforms.RandomRotation(15),
    transforms.RandomCrop(224, padding=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomAffine(degrees=20, scale=(0.8, 1.2), shear=10),
    transforms.RandomErasing(p=0.3),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  
])

transform_val_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [5]:
# Load the dataset
dataset = ImageFolder(root=dataset_dir, transform=transform_train)
print("Classes:", dataset.classes)
print("Class-to-Index Mapping:", dataset.class_to_idx)
print("Number of Samples:", len(dataset))

Classes: ['Fake', 'Real']
Class-to-Index Mapping: {'Fake': 0, 'Real': 1}
Number of Samples: 10826


In [5]:
#def get_model(model_name):
#    if model_name == "densenet121":
#        model = models.densenet121(pretrained=True)
#        # Modify the classifier for 2 output classes
#        model.classifier = nn.Linear(model.classifier.in_features, 2)
#        return model
def get_model(model_name):
    if model_name == "densenet121":
        # Load pre-trained DenseNet-121
        model = models.densenet121(pretrained=True)
        
        # Freeze all layers except the classifier
        for param in model.parameters():
            param.requires_grad = False
        
        # Unfreeze the fully connected (classifier) layers
        for param in model.classifier.parameters():
            param.requires_grad = True
        
        # Modify the classifier for 2 output classes (binary classification)
        model.classifier = nn.Linear(model.classifier.in_features, 2)
        
        return model


In [6]:
# Calculate metrics function
def calculate_metrics(model, loader, device):
    
    # Set the model to evaluation mode (disables dropout)
    model.eval()

    # Lists to store true labels and predicted labels
    all_labels = []
    all_predictions = []

    # Disabling gradient computation
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
             # Get predicted labels by taking the argmax (most likely class)
            _, predicted = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

     # Calculate the confusion matrix,which give TN, FP, FN, and TP
    conf_matrix = confusion_matrix(all_labels, all_predictions)
    # Unpack the confusion matrix into four components: TN, FP, FN, TP
    TN, FP, FN, TP = conf_matrix.ravel() 

    total = conf_matrix.sum()
    accuracy = (TP + TN) / total if total > 0 else 0.0
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0.0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    return accuracy, precision, recall, f1, conf_matrix

In [7]:
# Train the model function with validation accuracy printed after each epoch
def train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=5):
    # Variable to track the best validation accuracy
    best_val_accuracy = 0
    
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
         # Iterate over batches in the training data
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader)}")
        # Validation phase
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        val_accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{epochs}, Validation Accuracy: {val_accuracy:.2f}%")
        
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
    
    return best_val_accuracy

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Cross-validation setup
num_folds = 3
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)


In [8]:
def objective(trial, model_name):
    # Get a suggested learning rate from Optuna
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    
    # Initialize the model with dropout
    model = get_model(model_name).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    val_accuracies = []
    for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(dataset)):
        print(f"Fold {fold_idx + 1}/{num_folds}")
        
        # Create training/validation split
        train_val_data = Subset(dataset, train_val_idx)
        test_data = Subset(dataset, test_idx)
        
        train_size = int(0.8 * len(train_val_data))
        val_size = len(train_val_data) - train_size
        train_data, val_data = torch.utils.data.random_split(
            train_val_data, [train_size, val_size], generator=torch.Generator().manual_seed(42)
        )
        train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
        
        # Train the model and get validation accuracy
        train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=5)
        
        # Evaluate on validation set
        val_accuracy, _, _, _, _ = calculate_metrics(model, val_loader, device)
        val_accuracies.append(val_accuracy)
    
    # Return the average validation accuracy across all folds as the objective value
    return np.mean(val_accuracies)

In [9]:
def evaluate_test_set(model_name, best_lr):
    # Initialize model with the best learning rate
    model = get_model(model_name).to(device)
    optimizer = optim.Adam(model.parameters(), lr=best_lr)
    criterion = nn.CrossEntropyLoss()

    fold_metrics = []
    for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(dataset)):
        print(f"\nEvaluating on Fold {fold_idx + 1}/{num_folds}")
        
        # Create training/validation split
        train_val_data = Subset(dataset, train_val_idx)
        test_data = Subset(dataset, test_idx)
        
        train_size = int(0.8 * len(train_val_data))
        val_size = len(train_val_data) - train_size
        train_data, val_data = torch.utils.data.random_split(
            train_val_data, [train_size, val_size], generator=torch.Generator().manual_seed(42)
        )
        train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
        
        # Train the model
        train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=5)
        
        # Evaluate on the test set
        test_loader = DataLoader(test_data, batch_size=32, shuffle=False)
        fold_metrics.append(calculate_metrics(model, test_loader, device))
    
    # Print metrics for each fold
    for fold_idx, metrics in enumerate(fold_metrics):
        accuracy, precision, recall, f1, conf_matrix = metrics
        print(f"Fold {fold_idx + 1} Metrics:")
        print(f"Accuracy: {accuracy:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}, F1-Score: {f1:.2f}")
        print(f"Confusion Matrix:\n{conf_matrix}")

# Calculate average metrics across folds
    avg_accuracy = np.mean([metrics[0] for metrics in fold_metrics])
    avg_precision = np.mean([metrics[1] for metrics in fold_metrics])
    avg_recall = np.mean([metrics[2] for metrics in fold_metrics])
    avg_f1 = np.mean([metrics[3] for metrics in fold_metrics])
    total_conf_matrix = np.sum([metrics[4] for metrics in fold_metrics], axis=0)

    print("\nAverage Metrics Across Folds:")
    print(f"Accuracy: {avg_accuracy:.2f}, Precision: {avg_precision:.2f}, Recall: {avg_recall:.2f}, F1-Score: {avg_f1:.2f}")
    print(f"Confusion Matrix (sum of all folds):\n{total_conf_matrix}")

In [10]:
# Optuna Optimization and Final Testing
for model_name in ["densenet121"]:
    print(f"\nOptimizing for {model_name.upper()}...")
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(trial, model_name), n_trials=5)  

    # Best learning rate found for the model
    best_lr = study.best_params['lr']
    print(f"Best Learning Rate for {model_name.upper()}: {best_lr}")

    # Evaluate on test sets for each fold
    evaluate_test_set(model_name, best_lr)

[I 2024-12-25 03:33:27,749] A new study created in memory with name: no-name-42b87c05-09de-4088-8dc6-e05fd93c3908



Optimizing for DENSENET121...


<ipython-input-8-2e749100ab16>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /ro

Fold 1/3
Epoch 1/5, Loss: 0.7083361425452469
Epoch 1/5, Validation Accuracy: 50.42%
Epoch 2/5, Loss: 0.6887235483411926
Epoch 2/5, Validation Accuracy: 55.96%
Epoch 3/5, Loss: 0.6776838855848787
Epoch 3/5, Validation Accuracy: 57.34%
Epoch 4/5, Loss: 0.6646247285505684
Epoch 4/5, Validation Accuracy: 60.94%
Epoch 5/5, Loss: 0.6535120016962125
Epoch 5/5, Validation Accuracy: 62.05%
Fold 2/3
Epoch 1/5, Loss: 0.6493049995016656
Epoch 1/5, Validation Accuracy: 64.47%
Epoch 2/5, Loss: 0.6398751966202456
Epoch 2/5, Validation Accuracy: 65.93%
Epoch 3/5, Loss: 0.6351944109352913
Epoch 3/5, Validation Accuracy: 64.54%
Epoch 4/5, Loss: 0.6292336533741397
Epoch 4/5, Validation Accuracy: 65.79%
Epoch 5/5, Loss: 0.6197133610920352
Epoch 5/5, Validation Accuracy: 65.72%
Fold 3/3
Epoch 1/5, Loss: 0.6151255382358699
Epoch 1/5, Validation Accuracy: 66.83%
Epoch 2/5, Loss: 0.6181266838015772
Epoch 2/5, Validation Accuracy: 68.84%
Epoch 3/5, Loss: 0.6010530416478109
Epoch 3/5, Validation Accuracy: 68.35

[I 2024-12-25 04:03:29,439] Trial 0 finished with value: 0.6606648199445984 and parameters: {'lr': 2.8258362953509898e-05}. Best is trial 0 with value: 0.6606648199445984.
<ipython-input-8-2e749100ab16>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weig

...

Fold 1/3
Epoch 1/5, Loss: 0.8806112818625751
Epoch 1/5, Validation Accuracy: 65.44%
Epoch 2/5, Loss: 0.8294349294670379
Epoch 2/5, Validation Accuracy: 62.67%
Epoch 3/5, Loss: 0.7460428286981846
Epoch 3/5, Validation Accuracy: 67.87%
Epoch 4/5, Loss: 0.8407730504623434
Epoch 4/5, Validation Accuracy: 66.00%
Epoch 5/5, Loss: 0.7996202468542762
Epoch 5/5, Validation Accuracy: 64.68%
Fold 2/3
Epoch 1/5, Loss: 0.8825977110401702
Epoch 1/5, Validation Accuracy: 58.24%
Epoch 2/5, Loss: 0.8711639869937581
Epoch 2/5, Validation Accuracy: 70.22%
Epoch 3/5, Loss: 0.8320537331354553
Epoch 3/5, Validation Accuracy: 69.46%
Epoch 4/5, Loss: 0.7756334072318525
Epoch 4/5, Validation Accuracy: 70.50%
Epoch 5/5, Loss: 0.7614442984372871
Epoch 5/5, Validation Accuracy: 68.98%
Fold 3/3
Epoch 1/5, Loss: 0.8091335044710676
Epoch 1/5, Validation Accuracy: 72.37%
Epoch 2/5, Loss: 0.8653986704283656
Epoch 2/5, Validation Accuracy: 70.50%
Epoch 3/5, Loss: 0.9008888831125439
Epoch 3/5, Validation Accuracy: 68.91

[I 2024-12-25 05:32:06,462] Trial 3 finished with value: 0.677746999076639 and parameters: {'lr': 0.016711351365217328}. Best is trial 1 with value: 0.7024469067405356.
<ipython-input-8-2e749100ab16>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights

Fold 1/3
Epoch 1/5, Loss: 0.6864445453190672
Epoch 1/5, Validation Accuracy: 62.95%
Epoch 2/5, Loss: 0.6369747150668782
Epoch 2/5, Validation Accuracy: 65.24%
Epoch 3/5, Loss: 0.6216847922920522
Epoch 3/5, Validation Accuracy: 67.24%
Epoch 4/5, Loss: 0.6056085800268374
Epoch 4/5, Validation Accuracy: 68.49%
Epoch 5/5, Loss: 0.5905672417161214
Epoch 5/5, Validation Accuracy: 67.45%
Fold 2/3
Epoch 1/5, Loss: 0.5883290128813264
Epoch 1/5, Validation Accuracy: 69.53%
Epoch 2/5, Loss: 0.5836724711386538
Epoch 2/5, Validation Accuracy: 70.91%
Epoch 3/5, Loss: 0.5798054370432268
Epoch 3/5, Validation Accuracy: 69.39%
Epoch 4/5, Loss: 0.5720331683672594
Epoch 4/5, Validation Accuracy: 69.39%
Epoch 5/5, Loss: 0.5686594265929902
Epoch 5/5, Validation Accuracy: 70.57%
Fold 3/3
Epoch 1/5, Loss: 0.569178360601815
Epoch 1/5, Validation Accuracy: 70.84%
Epoch 2/5, Loss: 0.563130087450723
Epoch 2/5, Validation Accuracy: 71.81%
Epoch 3/5, Loss: 0.5607325036222763
Epoch 3/5, Validation Accuracy: 72.09%


[I 2024-12-25 06:01:51,635] Trial 4 finished with value: 0.713758079409049 and parameters: {'lr': 0.00015796033417859846}. Best is trial 4 with value: 0.713758079409049.
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Best Learning Rate for DENSENET121: 0.00015796033417859846

Evaluating on Fold 1/3
Epoch 1/5, Loss: 0.6648815860405811
Epoch 1/5, Validation Accuracy: 63.50%
Epoch 2/5, Loss: 0.6229810168071347
Epoch 2/5, Validation Accuracy: 67.52%
Epoch 3/5, Loss: 0.6050401977083301
Epoch 3/5, Validation Accuracy: 66.41%
Epoch 4/5, Loss: 0.585142982269519
Epoch 4/5, Validation Accuracy: 68.77%
Epoch 5/5, Loss: 0.5842451249038317
Epoch 5/5, Validation Accuracy: 69.04%

Evaluating on Fold 2/3
Epoch 1/5, Loss: 0.5803197024606209
Epoch 1/5, Validation Accuracy: 69.67%
Epoch 2/5, Loss: 0.5809921628862454
Epoch 2/5, Validation Accuracy: 70.43%
Epoch 3/5, Loss: 0.5697306195346031
Epoch 3/5, Validation Accuracy: 71.19%
Epoch 4/5, Loss: 0.5631525946256205
Epoch 4/5, Validation Accuracy: 70.71%
Epoch 5/5, Loss: 0.5676288919224923
Epoch 5/5, Validation Accuracy: 72.16%

Evaluating on Fold 3/3
Epoch 1/5, Loss: 0.5657112242767165
Epoch 1/5, Validation Accuracy: 71.68%
Epoch 2/5, Loss: 0.5637625612277353
Epoch 2/5

In [39]:
def initialize_model(name):
    if model_name == "densenet121":
        # Load DenseNet-121 without pre-trained weights (pretrained=False)
        model = models.densenet121(pretrained=True)
        
        # Accessing the last two convolutional layers by their names
        layer_list = list(model.features.children())  # Extract layers as a list
        
        # The last conv layers are within dense block 4's final part
        last_conv_layer = layer_list[-1]  # Last layer is the final 1x1 convolution before the classifier
        
        # Unfreeze parameters of the last two conv layers
        for param in last_conv_layer.parameters():
            param.requires_grad = True

        # Optionally, unfreeze the second last conv layer if needed
        second_last_conv_layer = layer_list[-2]
        for param in second_last_conv_layer.parameters():
            param.requires_grad = True

        # Modify the classifier to adjust for binary classification
        model.classifier = nn.Sequential(
            nn.Linear(model.classifier.in_features, 512),  # Reduce features from input size to 512
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(512, 2) 
        )

        # Print the number of trainable parameters
        def count_trainable_params(model):
            return sum(p.numel() for p in model.parameters() if p.requires_grad)

        print(f"Total trainable parameters: {count_trainable_params(model):,}")

    else:
        raise ValueError(f"Model '{name}' is not supported. Only 'densenet121' is implemented.")

    return model

In [40]:
# for fine tunning
def objective_after(trial, model_name):
    # Get a suggested learning rate from Optuna
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    
    # Initialize the model with dropout
    model = initialize_model(model_name).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    val_accuracies = []
    for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(dataset)):
        print(f"Fold {fold_idx + 1}/{num_folds}")
        
        # Create training/validation split
        train_val_data = Subset(dataset, train_val_idx)
        test_data = Subset(dataset, test_idx)
        
        train_size = int(0.8 * len(train_val_data))
        val_size = len(train_val_data) - train_size
        train_data, val_data = torch.utils.data.random_split(
            train_val_data, [train_size, val_size], generator=torch.Generator().manual_seed(42)
        )
        train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
        
        # Train the model and get validation accuracy
        train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=5)
        
        # Evaluate on validation set
        val_accuracy, _, _, _, _ = calculate_metrics(model, val_loader, device)
        val_accuracies.append(val_accuracy)
    
    # Return the average validation accuracy across all folds as the objective value
    return np.mean(val_accuracies)

In [41]:
def evaluate_test(model_name, best_lr):
    # Initialize model with the best learning rate
    model = initialize_model(model_name).to(device)
    optimizer = optim.Adam(model.parameters(), lr=best_lr)
    criterion = nn.CrossEntropyLoss()

    fold_metrics = []
    for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(dataset)):
        print(f"\nEvaluating on Fold {fold_idx + 1}/{num_folds}")
        
        # Create training/validation split
        train_val_data = Subset(dataset, train_val_idx)
        test_data = Subset(dataset, test_idx)
        
        train_size = int(0.8 * len(train_val_data))
        val_size = len(train_val_data) - train_size
        train_data, val_data = torch.utils.data.random_split(
            train_val_data, [train_size, val_size], generator=torch.Generator().manual_seed(42)
        )
        train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
        
        # Train the model
        train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=5)
        
        # Evaluate on the test set
        test_loader = DataLoader(test_data, batch_size=32, shuffle=False)
        fold_metrics.append(calculate_metrics(model, test_loader, device))
    
    # Print metrics for each fold
    for fold_idx, metrics in enumerate(fold_metrics):
        accuracy, precision, recall, f1, conf_matrix = metrics
        print(f"Fold {fold_idx + 1} Metrics:")
        print(f"Accuracy: {accuracy:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}, F1-Score: {f1:.2f}")
        print(f"Confusion Matrix:\n{conf_matrix}")

# Calculate average metrics across folds
    avg_accuracy = np.mean([metrics[0] for metrics in fold_metrics])
    avg_precision = np.mean([metrics[1] for metrics in fold_metrics])
    avg_recall = np.mean([metrics[2] for metrics in fold_metrics])
    avg_f1 = np.mean([metrics[3] for metrics in fold_metrics])
    total_conf_matrix = np.sum([metrics[4] for metrics in fold_metrics], axis=0)

    print("\nAverage Metrics Across Folds:")
    print(f"Accuracy: {avg_accuracy:.2f}, Precision: {avg_precision:.2f}, Recall: {avg_recall:.2f}, F1-Score: {avg_f1:.2f}")
    print(f"Confusion Matrix (sum of all folds):\n{total_conf_matrix}")

In [42]:
# Optuna Optimization and Final Testing
for model_name in ["densenet121"]:
    print(f"\nOptimizing for {model_name.upper()}...")
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective_after(trial, model_name), n_trials=5)  

    # Best learning rate found for the model
    best_lr = study.best_params['lr']
    print(f"Best Learning Rate for {model_name.upper()}: {best_lr}")

    # Evaluate on test sets for each fold
    evaluate_test(model_name, best_lr)

[I 2024-12-25 18:22:42,129] A new study created in memory with name: no-name-eea36710-88de-435a-9939-8c573dae8663



Optimizing for DENSENET121...
Total trainable parameters: 7,479,682


<ipython-input-40-fd9538108a4f>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)


Fold 1/3
Epoch 1/5, Loss: 0.5811830647083935
Epoch 1/5, Validation Accuracy: 59.49%
Epoch 2/5, Loss: 0.4624601235376537
Epoch 2/5, Validation Accuracy: 75.00%
Epoch 3/5, Loss: 0.4187596277308069
Epoch 3/5, Validation Accuracy: 79.09%
Epoch 4/5, Loss: 0.40130866504176543
Epoch 4/5, Validation Accuracy: 82.89%
Epoch 5/5, Loss: 0.37205399161214986
Epoch 5/5, Validation Accuracy: 82.27%
Fold 2/3
Epoch 1/5, Loss: 0.3754355005137828
Epoch 1/5, Validation Accuracy: 83.59%
Epoch 2/5, Loss: 0.36511331434407945
Epoch 2/5, Validation Accuracy: 75.48%
Epoch 3/5, Loss: 0.35123563800727464
Epoch 3/5, Validation Accuracy: 85.18%
Epoch 4/5, Loss: 0.3325019691201205
Epoch 4/5, Validation Accuracy: 85.04%
Epoch 5/5, Loss: 0.3308088981973532
Epoch 5/5, Validation Accuracy: 85.11%
Fold 3/3
Epoch 1/5, Loss: 0.32436175208065393
Epoch 1/5, Validation Accuracy: 86.43%
Epoch 2/5, Loss: 0.3135527540307019
Epoch 2/5, Validation Accuracy: 84.00%
Epoch 3/5, Loss: 0.31170905109762487
Epoch 3/5, Validation Accuracy:

[I 2024-12-25 19:00:58,193] Trial 0 finished with value: 0.8462603878116344 and parameters: {'lr': 0.0010218456511740415}. Best is trial 0 with value: 0.8462603878116344.
<ipython-input-40-fd9538108a4f>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weig

...

Total trainable parameters: 7,479,682
Fold 1/3
Epoch 1/5, Loss: 1.3106200879450003
Epoch 1/5, Validation Accuracy: 49.38%
Epoch 2/5, Loss: 0.6935215091837045
Epoch 2/5, Validation Accuracy: 50.62%
Epoch 3/5, Loss: 0.6938461371548268
Epoch 3/5, Validation Accuracy: 50.62%
Epoch 4/5, Loss: 0.694017800836932
Epoch 4/5, Validation Accuracy: 49.38%
Epoch 5/5, Validation Accuracy: 50.62%
Fold 2/3
Epoch 1/5, Loss: 0.6940463056880466
Epoch 1/5, Validation Accuracy: 50.00%
Epoch 2/5, Loss: 0.694515637271312
Epoch 2/5, Validation Accuracy: 50.00%
Epoch 3/5, Loss: 0.6937158716976314
Epoch 3/5, Validation Accuracy: 50.00%
Epoch 4/5, Loss: 0.6950352547577073
Epoch 4/5, Validation Accuracy: 50.00%
Epoch 5/5, Loss: 0.6939180664594661
Epoch 5/5, Validation Accuracy: 50.00%
Fold 3/3
Epoch 1/5, Loss: 0.6938688804431515
Epoch 1/5, Validation Accuracy: 48.41%
Epoch 2/5, Loss: 0.6941283667943754
Epoch 2/5, Validation Accuracy: 51.59%
Epoch 3/5, Loss: 0.6932613326041079
Epoch 3/5, Validation Accuracy: 51.59

[I 2024-12-25 20:54:46,318] Trial 3 finished with value: 0.5073868882733149 and parameters: {'lr': 0.020171740431524397}. Best is trial 2 with value: 0.9180517082179133.
<ipython-input-40-fd9538108a4f>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weigh

Total trainable parameters: 7,479,682
Fold 1/3
Epoch 1/5, Loss: 0.7189875620504769
Epoch 1/5, Validation Accuracy: 51.18%
Epoch 2/5, Loss: 0.6885512481078259
Epoch 2/5, Validation Accuracy: 55.75%
Epoch 3/5, Loss: 0.6881127551774293
Epoch 3/5, Validation Accuracy: 51.66%
Epoch 4/5, Loss: 0.6880653401764717
Epoch 4/5, Validation Accuracy: 50.07%
Epoch 5/5, Loss: 0.69197360059833
Epoch 5/5, Validation Accuracy: 53.25%
Fold 2/3
Epoch 1/5, Loss: 0.6919330292643763
Epoch 1/5, Validation Accuracy: 57.69%
Epoch 2/5, Loss: 0.6878300701057055
Epoch 2/5, Validation Accuracy: 54.36%
Epoch 3/5, Loss: 0.6886446996288405
Epoch 3/5, Validation Accuracy: 58.59%
Epoch 4/5, Loss: 0.6862459409961384
Epoch 4/5, Validation Accuracy: 57.62%
Epoch 5/5, Loss: 0.6870491310377806
Epoch 5/5, Validation Accuracy: 58.17%
Fold 3/3
Epoch 1/5, Loss: 0.690136407291033
Epoch 1/5, Validation Accuracy: 55.54%
Epoch 2/5, Loss: 0.6900319137625931
Epoch 2/5, Validation Accuracy: 56.58%
Epoch 3/5, Loss: 0.687779539197848
Epo

[I 2024-12-25 21:33:02,456] Trial 4 finished with value: 0.556555863342567 and parameters: {'lr': 0.002004817761946956}. Best is trial 2 with value: 0.9180517082179133.
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Best Learning Rate for DENSENET121: 9.020485241046237e-05
Total trainable parameters: 7,479,682

Evaluating on Fold 1/3
Epoch 1/5, Loss: 0.4564156552869312
Epoch 1/5, Validation Accuracy: 84.14%
Epoch 2/5, Loss: 0.32761735250936685
Epoch 2/5, Validation Accuracy: 84.28%
Epoch 3/5, Loss: 0.2730421853452427
Epoch 3/5, Validation Accuracy: 87.47%
Epoch 4/5, Loss: 0.24607015273518326
Epoch 4/5, Validation Accuracy: 87.60%
Epoch 5/5, Loss: 0.21116373607713873
Epoch 5/5, Validation Accuracy: 89.61%

Evaluating on Fold 2/3
Epoch 1/5, Loss: 0.23535682623228316
Epoch 1/5, Validation Accuracy: 91.97%
Epoch 2/5, Loss: 0.21303271992101194
Epoch 2/5, Validation Accuracy: 92.04%
Epoch 3/5, Loss: 0.18131489430178596
Epoch 3/5, Validation Accuracy: 91.62%
Epoch 4/5, Loss: 0.1899389770519997
Epoch 4/5, Validation Accuracy: 92.59%
Epoch 5/5, Loss: 0.16710293000738924
Epoch 5/5, Validation Accuracy: 91.00%

Evaluating on Fold 3/3
Epoch 1/5, Loss: 0.19069518445275765
Epoch 1/5, Validation Accuracy: 92.52%